In [4]:

import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import io
import torch.nn as nn

In [5]:
filepath_train_en = '../data/raw/train.en' 
filepath_train_fr = '../data/raw/train.fr'

In [6]:
try:
    en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    fr_tokenizer = get_tokenizer('spacy', language='fr_core_news_sm')
except OSError:
    print("chưa tải được gói ngôn ngữ!")

In [7]:
def yield_tokens(file_path, tokenizer):
    with io.open(file_path, encoding='utf-8') as f:
        for line in f:
            yield tokenizer(line.strip())

In [8]:

special_tokens = ['<unk>', '<pad>', '<sos>', '<eos>']


vocab_en = build_vocab_from_iterator(
    yield_tokens(filepath_train_en, en_tokenizer),
    min_freq=2,
    specials=special_tokens,
    max_tokens=10000     
)

In [9]:
vocab_en.set_default_index(vocab_en['<unk>'])

vocab_fr = build_vocab_from_iterator(
    yield_tokens(filepath_train_fr, fr_tokenizer),
    min_freq=2,
    specials=special_tokens,
    max_tokens=10000
)
vocab_fr.set_default_index(vocab_fr['<unk>'])

In [10]:
PAD_IDX = vocab_en['<pad>']
SOS_IDX = vocab_en['<sos>']
EOS_IDX = vocab_en['<eos>']

In [11]:
def text_transform(tokenizer, vocab, text):
    token_list = tokenizer(text.strip())
    index_list = [vocab[token] for token in token_list]
    return torch.tensor([SOS_IDX] + index_list + [EOS_IDX])

In [12]:
def collate_batch(batch):
    src_batch, trg_batch = [], []
    src_lens = []

    for src_sample, trg_sample in batch:
        # Biến đổi văn bản thô thành tensor số
        src_item = text_transform(en_tokenizer, vocab_en, src_sample)
        trg_item = text_transform(fr_tokenizer, vocab_fr, trg_sample)
        
        src_batch.append(src_item)
        trg_batch.append(trg_item)
        # Lưu lại độ dài thật của câu tiếng Anh (để dùng cho pack_padded_sequence)
        src_lens.append(len(src_item))

    # --- BẮT BUỘC: Sắp xếp batch theo độ dài giảm dần ---
    #
    # Lý do: PyTorch yêu cầu input của packing phải được sort trước
    zipped = list(zip(src_batch, trg_batch, src_lens))
    # Sắp xếp dựa trên src_lens (phần tử thứ 2 trong tuple) từ cao xuống thấp
    zipped.sort(key=lambda x: x[2], reverse=True)
    
    # Tách ngược trở lại thành các list riêng lẻ
    src_batch, trg_batch, src_lens = zip(*zipped)
    
    # Chuyển src_lens sang tensor
    src_lens = torch.tensor(src_lens)

    # --- PADDING: Điền thêm <pad> vào câu ngắn ---
    # padding_value=PAD_IDX: Điền số 1 vào chỗ trống
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, padding_value=PAD_IDX)

    return src_batch, trg_batch, src_lens

In [13]:
BATCH_SIZE = 64  #

# Đọc dữ liệu thô từ file vào list (để đưa vào DataLoader)
def read_raw_data(path_en, path_fr):
    with open(path_en, encoding='utf-8') as f_en, open(path_fr, encoding='utf-8') as f_fr:
        return list(zip(f_en, f_fr))


train_data = read_raw_data(filepath_train_en, filepath_train_fr)

train_loader = DataLoader(
    train_data, 
    batch_size=BATCH_SIZE, 
    collate_fn=collate_batch,
    shuffle=True # Nên xáo trộn dữ liệu khi train
)

In [14]:
print("\n=== KIỂM TRA DATALOADER (PADDING & PACKING) ===")
src, trg, src_len = next(iter(train_loader))

print(f"✅ Kích thước Source Batch: {src.shape}")
print(f"   (Dài nhất trong batch x Batch Size)")
print(f"✅ Kích thước Target Batch: {trg.shape}")
print(f"✅ Danh sách độ dài (đã sắp xếp giảm dần chưa?):")
print(src_len) # In ra xem có phải là 1 dãy số giảm dần không (VD: 20, 19, 15, 10...)


=== KIỂM TRA DATALOADER (PADDING & PACKING) ===
✅ Kích thước Source Batch: torch.Size([25, 64])
   (Dài nhất trong batch x Batch Size)
✅ Kích thước Target Batch: torch.Size([34, 64])
✅ Danh sách độ dài (đã sắp xếp giảm dần chưa?):
tensor([25, 25, 22, 22, 22, 21, 20, 19, 19, 18, 18, 18, 18, 17, 17, 17, 17, 17,
        17, 16, 16, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14,
        14, 14, 14, 14, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 12, 12, 12, 11,
        11, 11, 11, 11, 11, 10, 10, 10, 10,  9])


In [15]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        # Lưu các tham số
        self.hid_dim = hid_dim      # Kích thước trạng thái ẩn (512)
        self.n_layers = n_layers    # Số lớp LSTM (2)
        
        # 1. Embedding: Biến index thành vector dense
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        # 2. Dropout: Tránh overfitting
        self.dropout = nn.Dropout(dropout)
        
        # 3. LSTM: Xử lý chuỗi
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
    def forward(self, src, src_len):
        # src: [src len, batch size]
        # src_len: [batch size] (Độ dài thật của từng câu trong batch)
        
        # Bước 1: Qua Embedding và Dropout
        # embedded: [src len, batch size, emb dim]
        embedded = self.dropout(self.embedding(src))
        
        # Bước 2: PACKING (Kỹ thuật nén dữ liệu)
        # Giúp LSTM bỏ qua các số 0 (padding), chỉ tính toán trên từ thật
        #
        packed_embedded = pack_padded_sequence(embedded, src_len.to('cpu'), enforce_sorted=True)
        
        # Bước 3: Đưa qua LSTM
        # packed_outputs: Đầu ra dạng nén (chứa hidden state của tất cả các bước)
        # hidden: Trạng thái ẩn cuối cùng (Context Vector) [n layers, batch size, hid dim]
        # cell:   Trạng thái tế bào nhớ cuối cùng (Context Vector)
        packed_outputs, (hidden, cell) = self.rnn(packed_embedded)
        
        # (Tùy chọn) Nếu muốn lấy output dạng thường thì dùng pad_packed_sequence
        # outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_outputs)
        
        # Trả về Context Vector để Decoder sử dụng
        #
        return hidden, cell

In [16]:
# --- CẤU HÌNH THAM SỐ (Theo yêu cầu đề bài) ---
INPUT_DIM = len(vocab_en)  # Kích thước từ điển Anh (Khoảng 6191)
ENC_EMB_DIM = 256          #
HID_DIM = 512              #
N_LAYERS = 2               #
ENC_DROPOUT = 0.5          #

# Khởi tạo mô hình
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
print("✅ Đã khởi tạo Encoder:")
print(encoder)

# --- CHẠY THỬ VỚI 1 BATCH DỮ LIỆU ---
# Lấy 1 batch từ DataLoader đã làm ở bước trước
src, trg, src_len = next(iter(train_loader))

print(f"\n⏳ Đang test với Batch Size = {src.shape[1]}...")
print(f"   Độ dài câu dài nhất trong batch = {src.shape[0]}")

# Chạy Forward
# Lưu ý: Phải truyền cả src và src_len vào
hidden, cell = encoder(src, src_len)

print("\n=== KẾT QUẢ TEST ENCODER ===")
print(f"✅ Hidden State Shape: {hidden.shape}")
print(f"   (Mong đợi: [{N_LAYERS}, {src.shape[1]}, {HID_DIM}]) -> [2, 64, 512]")
print(f"✅ Cell State Shape:   {cell.shape}")
print(f"   (Mong đợi: [{N_LAYERS}, {src.shape[1]}, {HID_DIM}]) -> [2, 64, 512]")

if hidden.shape == (2, 64, 512):
    print("\n🎉 CHÚC MỪNG! Encoder hoạt động hoàn hảo.")
else:
    print("\n⚠️ Có gì đó sai sai về kích thước output.")

✅ Đã khởi tạo Encoder:
Encoder(
  (embedding): Embedding(6191, 256)
  (dropout): Dropout(p=0.5, inplace=False)
  (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
)

⏳ Đang test với Batch Size = 64...
   Độ dài câu dài nhất trong batch = 26

=== KẾT QUẢ TEST ENCODER ===
✅ Hidden State Shape: torch.Size([2, 64, 512])
   (Mong đợi: [2, 64, 512]) -> [2, 64, 512]
✅ Cell State Shape:   torch.Size([2, 64, 512])
   (Mong đợi: [2, 64, 512]) -> [2, 64, 512]

🎉 CHÚC MỪNG! Encoder hoạt động hoàn hảo.


In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        # Lưu các tham số
        self.output_dim = output_dim # Kích thước từ điển Pháp (khoảng 6555 từ)
        self.hid_dim = hid_dim       # 512
        self.n_layers = n_layers     # 2
        
        # 1. Embedding
        # Input là 1 từ đơn lẻ -> vector
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        # 2. LSTM
        # Input size = emb_dim
        # Hidden size = hid_dim
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        # 3. Linear (Lớp đầu ra)
        # Chuyển từ trạng thái ẩn (hid_dim) -> xác suất của từ tiếp theo (output_dim)
        # Công thức: p(y_t) = softmax(Linear(h_t))
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        # 4. Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input: [batch size] (Chỉ là 1 mảng chứa các từ hiện tại của mỗi câu trong batch)
        # hidden: [n layers, batch size, hid dim] (Context Vector từ bước trước)
        # cell:   [n layers, batch size, hid dim]
        
        # Bước 1: Thêm chiều Seq Len vào input
        # Vì LSTM yêu cầu input shape [Seq Len, Batch, Dim] mà ta chỉ chạy 1 bước
        # input: [1, batch size]
        input = input.unsqueeze(0)
        
        # Bước 2: Embedding & Dropout
        # embedded: [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))
        
        # Bước 3: Cho qua LSTM
        # output: [1, batch size, hid dim]
        # hidden, cell: Trạng thái mới để dùng cho bước sau
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        # Bước 4: Dự đoán từ tiếp theo
        # prediction: [batch size, output dim]
        # Loại bỏ chiều Seq Len bằng squeeze(0) trước khi đưa vào Linear
        prediction = self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [18]:
# --- CẤU HÌNH THAM SỐ DECODER ---
OUTPUT_DIM = len(vocab_fr) # Kích thước từ điển Pháp (6555)
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
DEC_DROPOUT = 0.5

# Khởi tạo Decoder
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
print("✅ Đã khởi tạo Decoder:")
print(decoder)

# --- CHẠY THỬ VỚI DỮ LIỆU GIẢ LẬP ---
# Giả sử ta đang ở bước đầu tiên của quá trình dịch
# Input: Lấy đại 1 batch chứa toàn token <sos> (Bắt đầu câu)
input_test = torch.tensor([vocab_fr['<sos>']] * 64) # Batch size 64

# Hidden, Cell: Lấy từ kết quả test Encoder lúc nãy (Context Vector)
# (Lưu ý: Phải chạy block test Encoder ở trên trước nhé)

print(f"\n⏳ Đang test Decoder 1 bước...")
prediction, hidden_new, cell_new = decoder(input_test, hidden, cell)

print("\n=== KẾT QUẢ TEST DECODER ===")
print(f"✅ Prediction Shape: {prediction.shape}")
print(f"   (Mong đợi: [Batch Size, Vocab Size]) -> [64, {OUTPUT_DIM}]")

print(f"✅ Hidden State Shape: {hidden_new.shape}")
print(f"   (Mong đợi: [Layers, Batch, Hid]) -> [2, 64, 512]")

if prediction.shape == (64, OUTPUT_DIM) and hidden_new.shape == (2, 64, 512):
    print("\n🎉 CHÚC MỪNG! Decoder hoạt động hoàn hảo.")
else:
    print("\n⚠️ Kích thước đầu ra chưa đúng, kiểm tra lại code.")

✅ Đã khởi tạo Decoder:
Decoder(
  (embedding): Embedding(6555, 256)
  (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
  (fc_out): Linear(in_features=512, out_features=6555, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

⏳ Đang test Decoder 1 bước...

=== KẾT QUẢ TEST DECODER ===
✅ Prediction Shape: torch.Size([64, 6555])
   (Mong đợi: [Batch Size, Vocab Size]) -> [64, 6555]
✅ Hidden State Shape: torch.Size([2, 64, 512])
   (Mong đợi: [Layers, Batch, Hid]) -> [2, 64, 512]

🎉 CHÚC MỪNG! Decoder hoạt động hoàn hảo.


In [19]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        # Kiểm tra kỹ thuật: Kích thước hidden của Encoder và Decoder phải khớp nhau
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, src_len, teacher_forcing_ratio=0.5):
        # src: [src len, batch size]
        # trg: [trg len, batch size]
        # src_len: [batch size] (Độ dài thật của câu nguồn)
        # teacher_forcing_ratio: Xác suất dùng từ thật để train (mặc định 0.5)
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # Tensor để lưu kết quả dự đoán (ban đầu toàn số 0)
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # Bước 1: Đưa cả câu qua Encoder để lấy Context Vector (hidden, cell)
        hidden, cell = self.encoder(src, src_len)
        
        # Bước 2: Bắt đầu giải mã (Decoder)
        # Token đầu tiên đưa vào Decoder luôn là <sos> (Start of Sentence)
        input = trg[0, :]
        
        # Vòng lặp chạy từng từ một: từ 1 đến hết câu
        for t in range(1, trg_len):
            
            # Chạy Decoder 1 bước
            # input: từ hiện tại
            # hidden, cell: trạng thái cũ -> cập nhật thành mới
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # Lưu kết quả dự đoán vào tensor outputs
            outputs[t] = output
            
            # --- TEACHER FORCING LOGIC ---
            # Quyết định xem có "nhắc bài" không?
            teacher_force = random.random() < teacher_forcing_ratio
            
            # Lấy từ có xác suất cao nhất mà máy vừa đoán được
            top1 = output.argmax(1) 
            
            # Nếu teacher_force = True -> lấy từ đúng trong tập trg (target)
            # Nếu False -> lấy từ máy vừa đoán (top1)
            input = trg[t] if teacher_force else top1
            
        return outputs

In [20]:
# --- 1. CẤU HÌNH THIẾT BỊ ---
# Nếu có GPU thì dùng, không thì dùng CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔹 Đang sử dụng thiết bị: {device}")

# --- 2. KHỞI TẠO CÁC MÔ HÌNH ---
# Encoder và Decoder (đã khai báo ở các bước trước)
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

# Mô hình tổng Seq2Seq
model = Seq2Seq(enc, dec, device).to(device)

print("✅ Đã khởi tạo thành công Seq2Seq Model!")

# --- 3. KHỞI TẠO TRỌNG SỐ (Tùy chọn nhưng nên làm) ---
# Giúp model học nhanh hơn bằng cách khởi tạo tham số ngẫu nhiên theo chuẩn
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)


# --- 4. CHẠY THỬ VỚI 1 BATCH ---
# Lấy 1 batch dữ liệu
src, trg, src_len = next(iter(train_loader))

# Chuyển dữ liệu sang thiết bị (CPU/GPU)
src = src.to(device)
trg = trg.to(device)
# src_len không cần .to(device) vì pack_padded_sequence cần nó ở CPU

print(f"\n⏳ Đang chạy thử Forward Pass...")
# model(src, trg, src_len)
output = model(src, trg, src_len)

print("\n=== KẾT QUẢ KIỂM TRA SEQ2SEQ ===")
print(f"✅ Output Shape: {output.shape}")
print(f"   (Mong đợi: [Trg Len, Batch Size, Vocab Size]) -> [{trg.shape[0]}, 64, {OUTPUT_DIM}]")

# Kiểm tra xem có lỗi chiều không
if output.shape == (trg.shape[0], 64, OUTPUT_DIM):
    print("\n🎉 TUYỆT VỜI! Mô hình đã sẵn sàng để huấn luyện.")
else:
    print("\n⚠️ Kích thước đầu ra chưa đúng.")

🔹 Đang sử dụng thiết bị: cpu
✅ Đã khởi tạo thành công Seq2Seq Model!

⏳ Đang chạy thử Forward Pass...

=== KẾT QUẢ KIỂM TRA SEQ2SEQ ===
✅ Output Shape: torch.Size([33, 64, 6555])
   (Mong đợi: [Trg Len, Batch Size, Vocab Size]) -> [33, 64, 6555]

🎉 TUYỆT VỜI! Mô hình đã sẵn sàng để huấn luyện.
